# Análise de Risco de Crédito - NuBank

Risco de crédito está associado à possibilidade de um cliente não cumprir com as obrigações contratuais, como hipotecas, dívidas de cartão de crédito e outros tipos de empréstimos.

Minimizar o risco de inadimplência é uma grande preocupação para instituições financeiras. Por esse motivo, bancos comerciais e de investimento, fundos de capital de risco, empresas de gestão de ativos e seguradoras, para citar alguns, estão cada vez mais contando com a tecnologia para prever quais clientes são mais propensos a não honrar com as suas dívidas.

Modelos de Machine Learning têm ajudado essas empresas a melhorar a precisão de suas análises de risco de crédito, fornecendo um método científico para identificar devedores em potencial com antecedência.

Neste projeto, construiremos um modelo para prever o risco de inadimplência do cliente para o Nubank, uma das maiores e importantes Fintechs brasileira.

## Importação das Bibliotecas

In [69]:
# import dos pacotes
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# definição do estilo estético das plotagens
sns.set_style()

# filtragem de warnings
import warnings
warnings.filterwarnings('ignore')   

from sklearn.impute import SimpleImputer

from sklearn.model_selection import train_test_split

# Set the option to display all columns
pd.set_option('display.max_columns', None)



## Importação do Dataset

In [70]:
df = pd.read_csv('data/acquisition_train.csv')

## Visualização e Entendimento dos Dados

In [71]:
df.head(1).T

,0
ids,343b7e7b-2cf8-e508-b8fd-0a0285af30aa
target_default,False
score_1,1Rk8w4Ucd5yR3KcqZzLdow==
score_2,IOVu8au3ISbo6+zmfnYwMg==
score_3,350.0
score_4,101.800832
score_5,0.259555
score_6,108.427273
risk_rate,0.4
last_amount_borrowed,25033.92


In [72]:
# Dataframe shape
print('Número de linhas do dataset: ', df.shape[0])
print('Número de colunas do dataset: ', df.shape[1])

Número de linhas do dataset:  45000
Número de colunas do dataset:  43


In [73]:
# Informações sobre o dataset	
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45000 entries, 0 to 44999
Data columns (total 43 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   ids                                               45000 non-null  object 
 1   target_default                                    41741 non-null  object 
 2   score_1                                           44438 non-null  object 
 3   score_2                                           44438 non-null  object 
 4   score_3                                           44438 non-null  float64
 5   score_4                                           45000 non-null  float64
 6   score_5                                           45000 non-null  float64
 7   score_6                                           45000 non-null  float64
 8   risk_rate                                         44438 non-null  float64
 9   last_amount_borro

- Podemos ver que algumas variáveis possuem valores ausentes. Vamos dar uma olhada mais de perto nelas.

In [74]:
# Porcentagem de valores faltantes
print(((df.isnull().sum() / df.shape[0]) * 100).sort_values(ascending=False))

target_fraud                                        96.617778
last_amount_borrowed                                66.568889
last_borrowed_in_months                             66.568889
ok_since                                            58.988889
external_data_provider_credit_checks_last_2_year    50.284444
external_data_provider_credit_checks_last_year      33.608889
credit_limit                                        30.666667
n_issues                                            25.653333
facebook_profile                                     9.906667
marketing_channel                                    7.951111
job_name                                             7.413333
target_default                                       7.242222
external_data_provider_email_seen_before             4.962222
lat_lon                                              3.028889
user_agent                                           1.604444
n_bankruptcies                                       1.548889
n_defaul

- Verificar as estatísticas descritivas das variáveis numéricas.

In [75]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
score_3,44438.0,3.479833e+02,110.429165,0.000000,270.000000,340.000000,420.000000,9.900000e+02
score_4,45000.0,1.000068e+02,3.178080,86.191572,97.867036,100.018900,102.141797,1.139782e+02
score_5,45000.0,5.000371e-01,0.288326,0.000035,0.252150,0.500753,0.748816,9.999734e-01
score_6,45000.0,9.992523e+01,10.016549,60.663039,93.198353,99.974796,106.640275,1.421924e+02
risk_rate,44438.0,2.967172e-01,0.101933,0.000000,0.220000,0.290000,0.360000,9.000000e-01
last_amount_borrowed,15044.0,1.378234e+04,8077.571933,1005.180000,7519.502500,12023.465000,19220.270000,3.505960e+04
last_borrowed_in_months,15044.0,4.176389e+01,10.252705,36.000000,36.000000,36.000000,36.000000,6.000000e+01
credit_limit,31200.0,3.438195e+04,36586.982218,0.000000,10000.000000,25647.000000,47086.750000,4.482690e+05
income,44438.0,7.160739e+04,51858.967748,4821.180000,45010.777500,61266.760000,86019.232500,5.000028e+06
ok_since,18455.0,3.506974e+01,21.663194,0.000000,17.000000,32.000000,50.000000,1.410000e+02


## Relatório Análise Inicial dos Dados

- O dataset possui **43** variáveis e **45.000** registros.
- Dessas **43** variáveis, **8** possuem mais de **10%** de valores missing, **target_fraud** é a variável com a maior porcentagem de valores ausentes **(96.6%)**, em segundo e terceiro lugar temos as variáveis **last_amount_borrowed** e **last_borrowed_in_months** com **(66.5%)** de valores ausentes.
- A variável **target_fraud** mostra se existe ou não fraude de cartão de crédito. Nesse caso essa variável não é importante para o projeto.Portanto, esta coluna será excluída.
- As variáveis **last_amount_borrowed**, **last_borrowed_in_months**, **ok_since** e **external_data_provider_credit_checks_last_2_year** possuem mais de 50% de valores ausentes, substituir esses valores é algo difícil, manter elas pode gerar erros durante a predição. Portanto, estas colunas serão excluídas.
- As variáveis **job_name**, **external_data_provider_first_name**, **profile_phone_number** e **zip** são variáveis insignificantes para o projeto, portanto serão excluídas.
- A variável **external_data_provider_email_seen_before** possui um valor mínimo de -999.000000, certamente foi um erro, nesse caso, vamos alterar para o tipo NaN.
- A variável **reported_income** possui um valor máximo descrito como inf(infinito), nesse caso, vamos alterar para o tipo NaN.
- A variável **credit_limit** possui um valor mínimo de 0.000000, isso não existe em instituições financeiras, é obrigatório liberar um valor X de crédito para o cliente. Portanto, este valor será substituído por NaN.
- As variáveis:
    - **ids**, **score_1**, **score_2**, **reason**, **facebook_profile**, **state**, **email**, **zip**, **channel**, **job_name**, **real_state**, **lat_lon**, **marketing_channel**, **profile_phone_number**, **shipping_zip_code**, **profile_tags**, **application_time_applied**, **external_data_provider_first_name**, **user_agent** e **target_fraud** são variáveis com valores categóricos não significantes para o projeto, portanto serão excluídas.

## Exclusão das Variáveis não significativas

In [76]:
# cópia do dataframe
df2 = df.copy()

In [77]:
exclude_columns = ["ids", "score_1", "score_2", "reason", "email",
                   "facebook_profile", "state", "zip", 
                   "channel", "job_name", "real_state", "lat_lon", "application_time_applied",
                   "marketing_channel", "profile_phone_number", "external_data_provider_first_name",
                   "shipping_zip_code", "profile_tags", "user_agent", "target_fraud"]

df2.drop(labels = exclude_columns, axis=1, inplace=True)

## Alteração da Variável reported_income com Valor Inf

In [79]:
df2.replace([np.inf, -np.inf], np.nan, inplace=True)

## Dividir os dados em treino e teste

- Antes de iniciar as transformações e pré-processamentos dos dados, é necessário dividir o dataset em treino e teste, mas porque?
    - Porque precisamos evitar o "Data Leakage". Data Leakage ocorre quando informações do conjunto de dados de teste ou validação vazam para o conjunto de treinamento durante o pré-processamento ou modelagem, ou quando informaçõpes do target vazam para as features. Nessas situações, o que vai acontecer é que você vai ver um modelo muito bom, mas isso será ilusório, pois o seu modelo "roubou" para ter o resultado bom.

- Referências sobre Data Leakage: 
    - https://www.linkedin.com/company/universidade-dos-dados/posts/?feedView=all
    - https://www.casadocodigo.com.br/products/livro-escd
    - https://estatsite.com.br/2020/12/12/data-leakage-o-erro-que-ate-os-grandes-cometem/


In [80]:
# Dividindo os dados primeiro

X = df2.drop(columns=['target_default'])	
y = df2['target_default']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [81]:
train = pd.concat([X_train, y_train], axis=1)
train.head()

,score_3,score_4,score_5,score_6,risk_rate,last_amount_borrowed,last_borrowed_in_months,credit_limit,income,ok_since,n_bankruptcies,n_defaulted_loans,n_accounts,n_issues,application_time_in_funnel,external_data_provider_credit_checks_last_2_year,external_data_provider_credit_checks_last_month,external_data_provider_credit_checks_last_year,external_data_provider_email_seen_before,external_data_provider_fraud_score,reported_income,shipping_state,target_default
24224,240.0,105.331178,0.893486,94.495216,0.26,NaN,NaN,21968.0,45013.96,NaN,1.0,0.0,11.0,11.0,278,0.0,2,NaN,12.0,741,92586.0,BR-ES,False
38081,270.0,103.211042,0.605574,101.567582,0.33,19237.32,36.0,40972.0,80022.23,NaN,0.0,0.0,12.0,12.0,141,0.0,3,1.0,57.0,659,95975.0,BR-GO,False
2955,260.0,102.557916,0.917576,117.057610,0.28,NaN,NaN,NaN,19225.52,NaN,0.0,0.0,7.0,NaN,114,NaN,1,NaN,54.0,905,53981.0,BR-PR,False
30973,300.0,101.664122,0.627588,89.140113,0.28,NaN,NaN,NaN,60043.78,62.0,NaN,0.0,9.0,NaN,484,0.0,2,NaN,20.0,603,140976.0,BR-DF,False
41258,350.0,109.603725,0.657344,98.574923,0.27,NaN,NaN,156549.0,45032.90,45.0,0.0,0.0,25.0,25.0,300,NaN,3,NaN,42.0,911,120129.0,BR-SP,False


## Input Missing

In [82]:
train_num = train.select_dtypes(exclude='object').columns
train_cat = train.select_dtypes(include='object').columns

In [86]:
# Input Missing Variáveis Numéricas

imputer = SimpleImputer(missing_values=np.nan, strategy='median')
imputer = imputer.fit(train.loc[:, train_num])
train.loc[:, train_num] = imputer.transform(train.loc[:, train_num])

# Input Missing Variáveis Categóricas

imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imputer = imputer.fit(train.loc[:, train_cat])
train.loc[:, train_cat] = imputer.transform(train.loc[:, train_cat])


In [89]:
train.isna().sum()

score_3                                             0
score_4                                             0
score_5                                             0
score_6                                             0
risk_rate                                           0
last_amount_borrowed                                0
last_borrowed_in_months                             0
credit_limit                                        0
income                                              0
ok_since                                            0
n_bankruptcies                                      0
n_defaulted_loans                                   0
n_accounts                                          0
n_issues                                            0
application_time_in_funnel                          0
external_data_provider_credit_checks_last_2_year    0
external_data_provider_credit_checks_last_month     0
external_data_provider_credit_checks_last_year      0
external_data_provider_email